In [1]:
import pathlib
import cv2,os, time
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
import numpy as np
import matplotlib.pyplot as plt
from typing import *
from IPython.display import clear_output

In [2]:
path = pathlib.Path('/media/ibex/ibex_hdd/Mastitis_Data/yeniCihaz/Mart-14-2023')

In [3]:
images = [str(p) for p in path.glob('*/*.png')]

In [4]:
images.sort(key = lambda x: int(x.split(os.sep)[-1].split('.')[0]))

In [5]:
class motion_detector():
    sensitivity:int
    threshold: int
    def __init__(self,sensitivity:int=100,threshold:int=30) -> None:
        self.sensitivity=sensitivity #px
        self.threshold = threshold
        
    def __call__(self, images:List[np.ndarray]) -> bool:
        threshold:np.ndarray
        motion_mask:np.ndarray
        # input should have 5 images at least
        images0:List[np.ndarray] = images[:-1]
        images1:List[np.ndarray] = images[1:]
        # Compute absolute difference between consecutive images
        diffs:List[np.ndarray] = [cv2.absdiff(i0,i1) for i0,i1 in zip(images0,images1)]
        # Threshold the differences to create binary motion masks
        thresholds:List[np.ndarray] = [cv2.threshold(diff, self.threshold, 255, cv2.THRESH_BINARY)[1] for diff in diffs ]
        # Perform bitwise OR operation to combine the motion masks
        motion_mask:List[np.ndarray] = cv2.bitwise_or(thresholds[0], thresholds[1])
        for threshold in thresholds[2:]:    
            motion_mask = cv2.bitwise_or(motion_mask, threshold)
        # Check if motion is detected
        if cv2.countNonZero(motion_mask) > self.sensitivity:
            return True
        return False

In [6]:
# Load the 5 images
pathes = images
every_n_image = 5
thresh = 30
font = cv2.FONT_HERSHEY_SIMPLEX
recorded = 0
moving = motion_detector(sensitivity=200,threshold=30)
for i in range(5,len(pathes)):
    if i%1000==0:
        print(i)
        print('recorded',recorded)
        print('not recorded',i-recorded)
    this_pathes = pathes[i-5:i]
    this_images=[]
    for p in this_pathes:
        if os.stat(p).st_size == 0: continue
        try:
            img = cv2.imread(p,0)
            img = cv2.rotate(img,cv2.ROTATE_90_COUNTERCLOCKWISE)
            this_images.append(img)
        except:
            pass
    if len(this_images)<3:
        continue
    isMoving = moving(this_images)
    img = this_images[2]
    # img = cv2.resize(src=img,dst=img,dsize=(120*3,160*3))
    if isMoving:
        recorded+=1
        # img = cv2.circle(img,(50,50),10,(255)*3,-1)

    

    # cv2.imshow('',img)
    # cv2.waitKey(10)
    # Display the motion mask
    # plt.figure(figsize=(100,60))

    # plt.imshow(vis)
    # plt.show()
    # time.sleep(1)
    # clear_output(wait=True)
# cv2.destroyAllWindows()
print('recorded',recorded)
print('not recorded',len(images)-recorded)

1000
recorded 295
not recorded 705
2000
recorded 889
not recorded 1111
3000
recorded 1255
not recorded 1745
4000
recorded 1819
not recorded 2181
5000
recorded 2226
not recorded 2774
6000
recorded 2837
not recorded 3163
7000
recorded 3662
not recorded 3338
8000
recorded 4169
not recorded 3831
9000
recorded 4647
not recorded 4353
10000
recorded 5221
not recorded 4779
11000
recorded 5469
not recorded 5531
12000
recorded 6178
not recorded 5822
13000
recorded 6853
not recorded 6147
14000
recorded 7398
not recorded 6602
15000
recorded 7555
not recorded 7445
16000
recorded 8070
not recorded 7930
17000
recorded 8558
not recorded 8442
18000
recorded 9169
not recorded 8831
19000
recorded 9633
not recorded 9367
20000
recorded 10018
not recorded 9982
21000
recorded 10533
not recorded 10467
22000
recorded 11329
not recorded 10671
23000
recorded 11959
not recorded 11041
24000
recorded 12206
not recorded 11794
25000
recorded 12640
not recorded 12360
26000
recorded 12988
not recorded 13012
27000
recor